<h3>
ΦC<sub>4</sub>H<sub>10</sub> + (6.5)(O<sub>2</sub> + 3.76 N<sub>2</sub>
) → a CO<sub>2</sub>+ b CO + d H<sub>2</sub>O + e H<sub>2</sub>+ f O<sub>2</sub>+g N<sub>2</sub>
</h3>
<hr>
P = 1 atm<br>
T = 298.15 K<br>
Φ=0.9

<h4>Equations:</h4>
Carbon Balance:
a+b=3.6<br>
b=3.6-a

Hydrogen Balance:
2*(d+e)=9<br>
d=4.5-e

Oxygen Balance: 
2(a+f)+b+d=13<br>
2(a+f)+(3.6-a)+(4.5-e)=13<br>
a+2*f-e=4.9<br>
f=2.45- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>

Nitrogen Balance:
g=24.44

<h3>
0.9C<sub>4</sub>H<sub>10</sub> + (6.5)(O<sub>2</sub> + 3.76 N<sub>2</sub>
) → a CO<sub>2</sub>+ (3.6-a) CO + (4.5-e) H<sub>2</sub>O + e H<sub>2</sub>+ (2.45- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>) O<sub>2</sub>+24.44 N<sub>2</sub>
</h3>

total number of moles N<sub>T</sub><br>
= a+(3.6-a)+(4.5-e)+e+(2.45- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>)+24.44<br>

=34.99- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>


Equations:

CO + H<sub>2</sub>O ⇌ CO<sub>2</sub> + H<sub>2</sub> .....1

H<sub>2</sub>O ⇌ H<sub>2</sub> + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub> .............2

CO<sub>2</sub> ⇌ CO + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub>  ............3

In [1]:
from sympy import Eq , solve , symbols , Function, exp, solveset
import pandas as pd

In [2]:
a,b,d,e,f,g=symbols('a b d:g',real=True)

In [3]:
#Carbon balance:
c_eqn=Eq(a+b,3.6)
b=(solve(c_eqn,b)[0])

#Hydrogen balance:
h_eqn=Eq(2*d+2*e,9)
d=(solve(h_eqn,d)[0])

#Oxygen balance:
o_eqn=Eq(2*a+b+d+2*f,13)
f=(solve(o_eqn,f)[0])

#Nitrogen balance:
n_eqn=Eq(2*g,6.5*3.76*2)
g=(solve(n_eqn,g)[0])

N_total=a+b+d+e+f+g

CO + H<sub>2</sub>O ⇌ CO<sub>2</sub> + H<sub>2</sub> .....1

In [4]:
kp1=(a*e)/(b*d)*(1/N_total)**(1+1-1-1)
kp1

a*e/((3.6 - a)*(9/2 - e))

H<sub>2</sub>O ⇌ H<sub>2</sub> + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub> .............2

In [5]:
kp2=e*f**0.5/(d) * (1/(N_total))**(1+0.5-1) 

kp2

1.56524758424985*e*(-0.204081632653061*a + 0.204081632653061*e + 1)**0.5*(1/(-0.5*a + 0.5*e + 34.99))**0.5/(9/2 - e)

CO<sub>2</sub> ⇌ CO + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub>  ............3

In [6]:
kp3=b*(f**0.5)/(a)*(1/N_total)**(1+0.5-1) #- exp(props['Kp3'])
kp3

1.56524758424985*(3.6 - a)*(-0.204081632653061*a + 0.204081632653061*e + 1)**0.5*(1/(-0.5*a + 0.5*e + 34.99))**0.5/a

In [7]:
#defining functions to be used 
def get_key(v,dictionary):
    for key, value in dictionary.items():
         if v == value:
            return key
        

def interpolate(temp,df):
     
    for t in df.index[1:]:
        if t<temp:
            t1=t
        
        elif t>temp:
            t2=t
            break
    
    y1=df.loc[t1] 
    y2=df.loc[t2]
    
    y= (y1)+(y2-y1)*(temp-t1)/(t2-t1)
    
    return y

In [8]:
data_dict={
    'h_f':[ None,None,None,0, 0 , -393520 , -241820,-110530,0],
    298: [-11.554,-92.208,-103.762,0,0,0,0,0,0], 
    500: [-4.9252,-52.691,-57.616,5912,6088,8314,6920,5929,5883],
    1000: [-0.366,-23.163,-23.529,21460,22707,33405,25978,21686,20686],
    1200: [0.3108,-18.182,-17.871,28108,29765,44484,34476,28426,26794],
    1400: [0.767,-14.609,-13.842,34936,36966,55907,43447,35338,33062],
    1600: [1.091,-11.921,-10.83,41903,44279,67580,52844,42384,39522],
    1800: [1.328,-9.826,-8.497,48982,51689,79442,62609,49522,46150],
    2000: [1.51,-8.145,-6.635,56141,59199,91450,72689,56739,52932],
    2100:[None,None,None,59399,62653,96882,76666,60052,55840],
    2200: [1.648,-6.768,-5.12,62872,66304,102696,81345,63562,59065],
    2300: [None,None,None,66345,69955,108511,86024,67072,62291],
    2400: [1.759,-5.619,-3.86,69818,73607,114325,90703,70582,65516],
    2500: [None,None,None,73291,77258,120140,95382,74093,68741],
    2600: [1.847,-4.648,-2.801,None,None,None,None,None,None]
}

'CO + H2O ⇌ CO2 + H2 .1'

'H2O ⇌ H2 + 1/2 O2 ...2'

'CO2 ⇌ CO + 1/2 O2 ...3'

ind=['Kp1','Kp2','Kp3','N2','O2','CO2','H2O','CO','H2']
data=pd.DataFrame(data_dict,index=ind)



data2=data.T
data2[data2.columns[:3]]=data2[data2.columns[:3]].interpolate()

In [9]:
temps={}

vals={kp1:['kp1'], kp2: ['kp2'] , a:['a'] , b:['b'] , d:['d'] , e:['e'] , f:['f'] , g:['g'] }

In [10]:
#Assume Temperature=2000 K
t=2100
#temps.append(t)
#Getting properties at t

if t in data2.index:                #making sure t doesn't need interpolation
    props=data2.loc[t]

else:
    props=interpolate(t,data2)
    
    
eq1=Eq(kp1,exp(props['Kp1']))
eq2=Eq(kp2,exp(props['Kp2']))
eq3=Eq(kp3,exp(props['Kp3']))    

In [11]:
n_e=solve(eq3,e)[0]
new_a=[]
for i in solve(eq2.subs(e,n_e)):
    if i<3.6:
        new_a.append(i)
if len(new_a)==1:
    new_a=new_a[0]

In [12]:
for i in vals.keys():
    vals[i].append(i.subs({a:new_a,e:n_e.subs(a,new_a)}))
    print(vals[i][0],': ',i.subs({a:new_a,e:n_e.subs(a,new_a)}))

kp1 :  0.206181176213425
kp2 :  0.000577674497785665
a :  3.53149161915489
b :  0.0685083808451137
d :  4.48207275975159
e :  0.0179272402484134
f :  0.693217810546764
g :  24.4400000000000


<h3>
Q = H<sub>P</sub> - H<sub>R</sub> = 0

H<sub>P</sub> = H<sub>R</sub>
</h3>    

In [13]:
h_f={'H2' : 0 , 'N2' : 0, 'O2' : 0 , 'CO' : -110530 , 'CO2' : -393520 , 'H2O' : -241820}
h_f

{'H2': 0, 'N2': 0, 'O2': 0, 'CO': -110530, 'CO2': -393520, 'H2O': -241820}

In [14]:
h_f_C4H10=-103850

phi=0.9

In [15]:
H_R=phi*(h_f_C4H10) + 6.5*(data2.loc['h_f']['O2']+data2.loc[298]['O2']) + g*(data2.loc['h_f']['N2']+data2.loc[298]['N2'])

H_P=(a*(data2.loc['h_f']['CO2']+props['CO2']) + b*(data2.loc['h_f']['CO']+props['CO']) +d*(data2.loc['h_f']['H2O']+props['H2O']) +e*(data2.loc['h_f']['H2']+props['H2']) + f*(data2.loc['h_f']['O2']+props['O2']) + g*(data2.loc['h_f']['N2']+props['N2'])).subs({a:1,e:1})

Q=H_P - H_R 

temps[t]=Q

In [16]:
temps

{2100: 748596.610000000}

In [16]:
q_pos=1000000000
q_neg=-1000000000
if Q>0: #decrease Temperature
    t-=100
    q_pos=Q
else: #higher Temperature
    t+=50
    q_neg=Q

In [ ]:
counter=0

In [ ]:
while counter<20 :          
    
    #Getting properties at t
    
    if t in data2.index:                #making sure t doesn't need interpolation
        props=data2.loc[t]

    else:
        props=interpolate(t,data2)


    eq1=Eq(kp1,exp(props['Kp1']))
    eq2=Eq(kp2,exp(props['Kp2']))
    eq3=Eq(kp3,exp(props['Kp3']))


    n_e=solve(eq3,e)[0]
    new_a=[]
    for i in solve(eq2.subs(e,n_e)):
        if i<3.6:
            new_a.append(i)
    if len(new_a)==1:
        new_a=new_a[0]

    for i in vals.keys():
        vals[i].append(i.subs({a:new_a,e:n_e.subs(a,new_a)}))
        print(vals[i][0],': ',i.subs({a:new_a,e:n_e.subs(a,new_a)}))
    print('')    
    H_P=(a*(data2.loc['h_f']['CO2']+props['CO2']) + b*(data2.loc['h_f']['CO']+props['CO']) +d*(data2.loc['h_f']['H2O']+props['H2O']) +e*(data2.loc['h_f']['H2']+props['H2']) + f*(data2.loc['h_f']['O2']+props['O2']) + g*(data2.loc['h_f']['N2']+props['N2'])).subs({a:1,e:1})

    Q=H_P - H_R
    temps[t]=Q
    
    counter+=1    
    if Q>0 and Q<q_pos: #decrease Temperature
        t-=100
        q_pos=Q
    elif Q<0 and Q>q_neg: #higher Temperature
        t+=50
        q_neg=Q
    
    if q_pos!=1000000000 and q_neg!=-1000000000:
        t=(get_key(q_neg,temps))+(-q_neg)*(get_key(q_pos,temps)-get_key(q_neg,temps))/(q_pos-q_neg)

    